#N-gram probabilities and n-gram text generation
Samuel Peter (samuel.peter.25@dartmouth.edu)<br>
Dartmouth College, LING48, Spring 2024

Documentation of the NLTK.LM package:<br>
https://www.nltk.org/api/nltk.lm.html

Quick tip: How to extract n-gram probabilities:<br>
https://stackoverflow.com/questions/54962539/how-to-get-the-probability-of-bigrams-in-a-text-of-sentences

In [ ]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 770.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=2eeba2a949e78f05f0756b2e7ff7e39acda9b673f738c486d0816711d2a1d818
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstallin

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
import requests
import io
import random
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm import MLE, NgramCounter, Vocabulary
from nltk.util import ngrams
from collections import Counter
from nltk import word_tokenize, sent_tokenize, bigrams, trigrams
import gdown
from googletrans import Translator
from collections import Counter
from google.colab import files
import re

In [ ]:
# Download and decompress corpora from the homework
url = "https://drive.google.com/uc?id=1DAkd5C7HRTy0Tv2nSIWdpa4PMcKe5yZi"
output = 'hw4-corpora-2024.zip'
gdown.download(url, output, quiet=False)
!unzip -j hw4-corpora-2024.zip

Downloading...
From (original): https://drive.google.com/uc?id=1DAkd5C7HRTy0Tv2nSIWdpa4PMcKe5yZi
From (redirected): https://drive.google.com/uc?id=1DAkd5C7HRTy0Tv2nSIWdpa4PMcKe5yZi&confirm=t&uuid=b1cbf4b9-f5e5-433b-87ae-8808e464a927
To: /content/hw4-corpora-2024.zip
100%|██████████| 31.2M/31.2M [00:01<00:00, 24.6MB/s]


Archive:  hw4-corpora-2024.zip
  inflating: amharic-converted.txt   
  inflating: arabic-nawal-sadawi.txt  
  inflating: bangla-wiki.txt         
  inflating: english-shakespeare.txt  
  inflating: english-sherlock.txt    
  inflating: french-victor-hugo.txt  
  inflating: german-kafka.txt        
  inflating: greek-europarl-greek.txt  
  inflating: gujarati-ai4bharat.txt  
  inflating: hindi-jansatta-utf8.txt  
  inflating: igbo-corpus.txt         
  inflating: indonesian-wikipedia-sentences.txt  
  inflating: japanese-natsume-soseki.txt  
  inflating: kinyarwanda-corpus.txt  
  inflating: korean-news.txt         
  inflating: latin-virgil.txt        
  inflating: mandarin-lu-xun.txt     
  inflating: marathi-ai4bharat.txt   
  inflating: mauritian-creole-corpus.txt  
  inflating: navajo-wikipedia-10k.txt  
  inflating: nepali-artha-banijya.txt  
  inflating: norwegian-bokmal-sigrid-undset.txt  
  inflating: odia-ai4bharat.txt      
  inflating: polish-europarl-polish.txt  
  inflatin

In [ ]:
# Open file
#file = io.open('english-shakespeare.txt', encoding='utf8')
file = io.open('german-kafka.txt', encoding='utf8')
text = file.read()

# Preprocess the tokenized text for language modelling
https://www.nltk.org/api/nltk.lm.html

In [ ]:
def translate_to_english(text):
    # Initialize the translator
    translator = Translator()

    # Detect the language of the input text
    lang = translator.detect(text).lang

    # If the detected language is not English, translate the text
    if lang != 'en':
        translation = translator.translate(text, src=lang, dest='en')
        translated_text = translation.text
    else:
        # If the detected language is already English, return the text as is
        translated_text = text

    return translated_text

#Generate a 100 word texts for unigram, bigram, trigram, quadgram models

In [ ]:
# Preprocess the tokenized text for language modelling
n = 1
paddedLine = [list(pad_both_ends(word_tokenize(text.lower()), n))]
train, vocab = padded_everygram_pipeline(n, paddedLine)

# Train a n-gram maximum likelihood estimation model.
unimodel = MLE(n)
unimodel.fit(train, vocab)

#Generate Sequence
unigram_gen = unimodel.generate(100, random_seed=42)
unigram_seq = ' '.join(unigram_gen)
print("=== Generated text: unigram ===")
print(unigram_seq)

#Generate sequence translation to english
english_uni = translate_to_english(unigram_seq)
print("=== English translation: unigram ===")
print(english_uni)

=== Generated text: unigram ===
mir , den bitte schon nicht waren , forschem , beugte hitze , aussichtspunkt mode in bilder kindern theater , suchen oder du als zu draußen , . uns lachte tag schlaf ihre « er in und lärm verstand k. persönlicheres , brechen der , da . denn mich einer einvernehmungen beine den wächter mit lebhaft angelegenheit schien an er » mir irgendeine noch und sie brunelda , die den bekräftigte zeit vor die müde ersoffen wie gesellschaft dem das ist dem katze weg es bewegungen – hätte , , . mannes sondern fort , er übliche ihm « verschiedener
=== English translation: unigram ===
I, the request, were not already, research, bowed heat, viewpoint fashion in pictures children theater, search or you as too outside ,.We laughed at day sleep and noise k.more personal, break the, there.Because I am happy to have the guard with lively matter.Mannes but away, he uses “various


In [ ]:
# Preprocess the tokenized text for language modelling
n = 2
paddedLine = [list(pad_both_ends(word_tokenize(text.lower()), n))]
train, vocab = padded_everygram_pipeline(n, paddedLine)

# Train a n-gram maximum likelihood estimation model.
bimodel = MLE(n)
bimodel.fit(train, vocab)

#Generate Sequence
bigram_gen = bimodel.generate(100, random_seed=42)
bigram_seq = ' '.join(bigram_gen)
print("=== Generated text: bigram ===")
print(bigram_seq)

#Generate sequence translation to english
english_bi = translate_to_english(bigram_seq)
print("=== English translation: bigram ===")
print(english_bi)

=== Generated text: bigram ===
mir , desto deutlicher zu schmal wie das hielt , den kirchendiener , daß ich kein elegantes kleid und aas mißachten sie. « , zeigte auf alle andern verantwortung für seine sache ist zogen sich jemand sonst nicht viel niedriger beamter , denn die augenbrauen drang , die nur ein ihm dadurch darf sich nehmen ihnen auch nur dann förmlich zusammengewachsene felsenharte schutthaufen zu und so erbärmlich. « , der erde ununterscheidbar von der onkel eine von einzelheiten den er ließ aber kannst nicht etwa den übertriebenen mitleid , als daß ich sie hob , er überdies immer zugänglich sein
=== English translation: bigram ===
me, the more clearly too narrow as that held, the church servant that I am not an elegant dress and AAS displaced them.«, Pointed to all other responsibilities for his cause, someone else did not pull out much lower, because the eyebrows that only came up with one of them can only take them up to them and so grown -hard rubble heaps and so path

In [ ]:
# Preprocess the tokenized text for language modelling
n = 3
paddedLine = [list(pad_both_ends(word_tokenize(text.lower()), n))]
train, vocab = padded_everygram_pipeline(n, paddedLine)

# Train a n-gram maximum likelihood estimation model.
trimodel = MLE(n)
trimodel.fit(train, vocab)

#Generate Sequence
trigram_gen = trimodel.generate(100, random_seed=42)
trigram_seq = ' '.join(trigram_gen)
print("=== Generated text: trigram ===")
print(trigram_seq)

#Generate sequence translation to english
english_tri = translate_to_english(trigram_seq)
print("=== English translation: trigram ===")
print(english_tri)

=== Generated text: trigram ===
mir , deinetwegen will ich nicht widerstehen . nicht alle bestraft , es hervorzubringen , selbst dir gegenüber verhalten ? « » gewiß , wohin du fährst , hinter dieser vornehmen gesellschaft unbemerkt hinauszukommen . schon steht sie da nicht wie , denkt blumfeld , der aus der parfümflasche . » du fährst mit « , sagte k. , rümpfte die lippen lange liegen . ich fühlte an meiner lage nichts . wenn es notwendig ist . frische fülle . quellende wasser . » ich gehe überhaupt schon ins gekröse greifen und würde hier – natürlich nicht mehr zurückhalten und
=== English translation: trigram ===
I don't want to resist me because of your sake.Not all punished to produce it, behave towards you?"" Certainly where you go to get out behind this noble company unnoticed.Already she is not like, Blumfeld thinks that from the perfume bottle."You are driving," said K., wrinkled the lips for a long time.I didn't feel anything about my location.if it's necessary .Fresh abundanc

In [ ]:
# Preprocess the tokenized text for language modelling
n = 4
paddedLine = [list(pad_both_ends(word_tokenize(text.lower()), n))]
train, vocab = padded_everygram_pipeline(n, paddedLine)

# Train a n-gram maximum likelihood estimation model.
quadmodel = MLE(n)
quadmodel.fit(train, vocab)

#Generate Sequence
quadgram_gen = quadmodel.generate(100, random_seed=42)
quadgram_seq = ' '.join(quadgram_gen)
print("=== Generated text: quadgram ===")
print(quadgram_seq)

#Generate sequence translation to english
english_quad = translate_to_english(quadgram_seq)
print("=== English translation: quadgram ===")
print(english_quad)

=== Generated text: quadgram ===
mir , deinetwegen will ich fort ; weil ich mich an einer eisernen klammer festhielt , die oben in der luft herumschwingen können . jetzt aber , da sie nicht immerfort hinken kann , erfindet sie etwas anderes , etwas eigentlich wunderbares , die speise fiel nicht , sondern gingen zu dem nebenlift und blieben dort , sich umschlungen haltend . herr samsa nickte ihm bloß mehrmals kurz mit großen augen zu . daraufhin ging der herr tatsächlich sofort mit langen schritten ins vorzimmer ; seine beiden freunde hatten schon ein weilchen vorher , zu ertasten gesucht . barnabas gab
=== English translation: quadgram ===
I want to go away because of you;Because I held on to an iron bracket that can swing in the air at the top.But now, since she cannot always lag, she invents something else, something really wonderful, the food did not fell, but went to the side lift and stayed there.Mr. Samsa just briefly nodded him with big eyes several times.Then the gentleman actu

#Ngram Evaluation
The performance of n-gram models can be evaluated based on the coherence, naturalness, and understandability of the generated text. Typically, higher n-gram models (e.g., n = 3 or n = 4) capture more context and dependencies, leading to more coherent and understandable text compared to lower n-gram models (e.g., n = 1 or n = 2).

The higher n-gram models performed better because it can better capture long-range dependencies and context in the text, resulting in more logical and coherent output. Lower n-gram models produced "bad" output by generating text that lacks context, coherence, and naturalness, leading to fragmented or nonsensical sentences.

##Probabilites and Counts

###Code to find the most common trigrams

In [ ]:
import nltk
from nltk import word_tokenize
from collections import Counter
import re

# Download NLTK resources (you only need to run this once)
nltk.download('punkt')

def find_most_common_trigrams(file_path, num_trigrams):
    # Read the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Tokenize the text into words
    words = word_tokenize(text)

    # Filter out special characters and numbers
    words = [word for word in words if re.match(r'^[a-zA-Z]+$', word)]

    # Generate trigrams from the words
    trigrams = [(words[i], words[i+1], words[i+2]) for i in range(len(words)-2)]

    # Count the occurrences of each trigram
    trigram_counts = Counter(trigrams)

    # Filter out trigrams with special characters
    filtered_trigrams = [(trigram, count) for trigram, count in trigram_counts.items()
                         if all(word.isalpha() for word in trigram)]

    # Find the most common trigrams
    most_common_trigrams = sorted(filtered_trigrams, key=lambda x: x[1], reverse=True)[:num_trigrams]

    return most_common_trigrams

# Example usage:
from google.colab import files

# Upload your text file to Google Colab
#uploaded = files.upload()

# Get the file name
#file_name = next(iter(uploaded))
file_name = file.name
print(file_name)

# Specify the number of most common trigrams you want to find
num_trigrams = 10  # Change this number as needed

# Find the most common trigrams without special characters
most_common_trigrams = find_most_common_trigrams(file_name, num_trigrams)
print(f"Most common {num_trigrams} trigrams without special characters:")
for trigram, count in most_common_trigrams:
    print(trigram, "-", count)


german-kafka.txt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Most common 10 trigrams without special characters:
('hie', 'und', 'da') - 97
('in', 'der', 'Nacht') - 75
('mit', 'der', 'Hand') - 59
('in', 'der', 'Hand') - 51
('auf', 'und', 'ab') - 47
('hin', 'und', 'her') - 46
('Es', 'war', 'ein') - 46
('sagte', 'die', 'Wirtin') - 46
('sagte', 'der', 'Mann') - 44
('sagte', 'der', 'Maler') - 44


In [ ]:
def count_ngrams_from_text(text, n):
    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text into words
    words = re.findall(r'\b\w+\b', text)

    # Generate n-grams
    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]

    # Count the occurrences of each n-gram
    ngram_counts = Counter(ngrams)

    return ngram_counts

In [ ]:
# Specify the n-grams you want to count
unigram_counts = count_ngrams_from_text(text, 1)
bigram_counts = count_ngrams_from_text(text, 2)
trigram_counts = count_ngrams_from_text(text, 3)

# Count occurrences of specific n-grams
#phrase = "I pray you"
phrase = "in der Nacht"
unigram = phrase.lower().split()[0]  # Convert to lowercase for consistent matching
bigram = " ".join(phrase.lower().split()[:2])  # Convert to lowercase and form bigram
trigram = " ".join(phrase.lower().split())  # Convert to lowercase and form trigram

unigram_count = unigram_counts.get((unigram,), 0)
bigram_count = bigram_counts.get(tuple(bigram.split()), 0)
trigram_count = trigram_counts.get(tuple(trigram.split()), 0)

print("=== Counts ===")
print("Unigram count (", unigram, "): ", unigram_count)
print("Bigram count (", bigram, "): ", bigram_count)
print("Trigram count (", trigram, "): ", trigram_count)

=== Counts ===
Unigram count ( in ):  5816
Bigram count ( in der ):  1100
Trigram count ( in der nacht ):  76


In [ ]:
# Get words to check
word_to_check = trigram.split()

# Get the probability of each word in the trigram model
word_prob_unigram = trimodel.score(word_to_check[0])
word_prob_bigram = trimodel.score(word_to_check[1], context=[word_to_check[0]])
word_prob_trigram = trimodel.score(word_to_check[2], context=[word_to_check[0], word_to_check[1]])

print("=== Probabilities ===")
print("Unigram Probability (", word_to_check[0], "):", word_prob_unigram)
print("Bigram Probability (", word_to_check[0], word_to_check[1], "):", word_prob_bigram)
print("Trigram Probability (", trigram, "):", word_prob_trigram)

=== Probabilities ===
Unigram Probability ( in ): 0.010404864314340383
Bigram Probability ( in der ): 0.1892310338895579
Trigram Probability ( in der nacht ): 0.06909090909090909
